In [ ]:
!conda install pygame -y


In [ ]:
!conda install math -y


In [2]:
import pygame
import math

# Initialize pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 800, 600

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# Create screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Autonomous Parking Control")

# Clock for controlling frame rate
clock = pygame.time.Clock()

# Car attributes
car_width, car_height = 50, 30
car_x, car_y = 100, 500
car_angle = 0
car_speed = 0

# Parking spot attributes
parking_spot_x, parking_spot_y = 600, 200
parking_spot_width, parking_spot_height = 60, 100

# Obstacles
obstacles = [
    pygame.Rect(300, 300, 50, 50),
    pygame.Rect(450, 400, 50, 50),
]

def draw_car(x, y, angle):
    """Draws the car on the screen."""
    car_surface = pygame.Surface((car_width, car_height), pygame.SRCALPHA)
    car_surface.fill(BLUE)
    rotated_car = pygame.transform.rotate(car_surface, angle)
    rect = rotated_car.get_rect(center=(x, y))
    screen.blit(rotated_car, rect.topleft)

def autonomous_parking():
    """Controls the car's autonomous parking logic."""
    global car_x, car_y, car_angle, car_speed

    # Calculate the angle and distance to the parking spot
    dx = (parking_spot_x + parking_spot_width / 2) - car_x
    dy = (parking_spot_y + parking_spot_height / 2) - car_y
    target_angle = math.degrees(math.atan2(-dy, dx))
    distance = math.sqrt(dx**2 + dy**2)

    # Stop if the car's center is at the center of the parking spot
    car_center_x, car_center_y = car_x, car_y
    parking_center_x = parking_spot_x + parking_spot_width / 2
    parking_center_y = parking_spot_y + parking_spot_height / 2

    if (
        abs(car_center_x - parking_center_x) < 5
        and abs(car_center_y - parking_center_y) < 5
    ):
        car_speed = 0
        return

    # Avoid obstacles
    for obstacle in obstacles:
        obstacle_center_x = obstacle.x + obstacle.width / 2
        obstacle_center_y = obstacle.y + obstacle.height / 2
        obstacle_dx = obstacle_center_x - car_x
        obstacle_dy = obstacle_center_y - car_y
        obstacle_distance = math.sqrt(obstacle_dx**2 + obstacle_dy**2)

        if obstacle_distance < 100:  # If an obstacle is near
            avoidance_angle = math.degrees(math.atan2(-obstacle_dy, obstacle_dx))
            angle_diff = (avoidance_angle - car_angle) % 360
            if angle_diff > 180:
                angle_diff -= 360

            if angle_diff > 0:
                car_angle -= 2  # Turn left to avoid
            else:
                car_angle += 2  # Turn right to avoid

            car_x += 2 * math.cos(math.radians(car_angle))
            car_y -= 2 * math.sin(math.radians(car_angle))
            return  # Skip normal movement to handle obstacle avoidance

    # Adjust car's angle to align with the parking spot
    angle_diff = (target_angle - car_angle) % 360
    if angle_diff > 180:
        angle_diff -= 360

    if angle_diff > 2:
        car_angle += 2
    elif angle_diff < -2:
        car_angle -= 2

    # Move the car forward if it is not at the parking spot
    if distance > 5:
        car_speed = 2
        car_x += car_speed * math.cos(math.radians(car_angle))
        car_y -= car_speed * math.sin(math.radians(car_angle))
    else:
        car_speed = 0

def check_collision():
    """Checks if the car collides with any obstacles."""
    car_rect = pygame.Rect(
        car_x - car_width // 2, car_y - car_height // 2, car_width, car_height
    )
    for obstacle in obstacles:
        if car_rect.colliderect(obstacle):
            return True
    return False

# Main loop
running = True
while running:
    screen.fill(WHITE)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Draw parking spot
    pygame.draw.rect(
        screen, GREEN, (parking_spot_x, parking_spot_y, parking_spot_width, parking_spot_height), 2
    )

    # Draw obstacles
    for obstacle in obstacles:
        pygame.draw.rect(screen, RED, obstacle)

    # Autonomous parking logic
    autonomous_parking()

    # Check for collisions
    if check_collision():
        print("Collision detected!")
        running = False

    # Draw the car
    draw_car(car_x, car_y, car_angle)

    # Update the display
    pygame.display.flip()

    # Cap the frame rate
    clock.tick(30)

pygame.quit()


pygame 2.2.0 (SDL 2.30.10, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
